In [1]:
import pandas as pd
import numpy as np

In [2]:
X_train = pd.read_csv('X_train.csv')
y_train = pd.read_csv('y_train.csv')
X_test = pd.read_csv('X_test.csv')

In [3]:
del y_train['id']

In [4]:
import feature_process_helper

In [6]:
X_train, X_test = feature_process_helper.dates(X_train, X_test)

In [8]:
X_train, X_test = feature_process_helper.construction(X_train, X_test)

In [9]:
X_train, X_test = feature_process_helper.bools(X_train, X_test)

In [10]:
X_train, X_test = feature_process_helper.locs(X_train, X_test)

In [11]:
X_train, X_test = feature_process_helper.removal(X_train, X_test)

In [ ]:
X_train, X_test = feature_process_helper.dummies(X_train, X_test)

In [16]:
print np.shape(X_train)
print np.shape(X_test)

(59400, 35)
(14850, 35)


In [ ]:
import pickle
import os

In [ ]:
pickle.dump(X_train, open(os.path.join('pickles', 'X_train1.pkl'),'wb'), protocol=4)

In [ ]:
pickle.dump(X_test, open('X_test1.pkl','wb'), protocol=4)

In [ ]:
X_train = pickle.load(X_train1.pkl)

In [ ]:
X_test = pickle.load(X_test1.pkl)

### Run Random Forest classifier

In [20]:
from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier(criterion='entropy',
                                n_estimators=100,
                                max_features='auto',
                                oob_score=True,
                                random_state=1,
                                n_jobs=-1)
                            
rf.fit(X_train, y_train.values.ravel())
print "%.4f" % rf.oob_score_ 

0.8077


In [159]:
from sklearn.model_selection import cross_val_score

scores = cross_val_score(rf, X_train, y_train.values.ravel())
scores.mean()

0.80099326599326603

### Generate submission file

In [161]:
predictions = rf.predict(X_test)
y_test = pd.read_csv('y_test.csv')
pred = pd.DataFrame(predictions, columns = [y_test.columns[1]])
del y_test['status_group']
y_test = pd.concat((y_test, pred), axis = 1)
y_test.to_csv('y_test.csv', sep=",", index = False)

In [674]:
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier

In [850]:
from sklearn.ensemble import AdaBoostClassifier

tree = DecisionTreeClassifier(criterion='entropy', 
                              max_depth=1,
                              random_state=0)

ada = AdaBoostClassifier(base_estimator=tree,
                         n_estimators=100, 
                         learning_rate=.5,
                         random_state=0)

In [851]:
scores = cross_val_score(ada, X_train, y_train.values.ravel())

In [852]:
scores.mean()

0.73193602693602688

In [786]:
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression

param_range = [0.01, 0.1, 1.0, 10.0, 100.0]

param_grid = [{'C': param_range}]

gs = GridSearchCV(estimator=LogisticRegression(random_state=0),
                  param_grid=param_grid,
                  scoring='accuracy',
                  cv=2,
                  n_jobs=-1)

scores = cross_val_score(gs, X_train, y_train.values.ravel(), scoring='accuracy', cv=5)
print('CV accuracy: %.3f +/- %.3f' % (np.mean(scores), np.std(scores)))

CV accuracy: 0.747 +/- 0.004


In [27]:
nonbi = []
for i in X_train.columns:
    if X_train[i].nunique() > 2:
        nonbi.append(i)

for i in nonbi:
    del X_train[i]

In [33]:
from sklearn.naive_bayes import BernoulliNB

nb = BernoulliNB()

scores = cross_val_score(nb, X_train, y_train.values.ravel())
scores.mean()

0.70023569023569021